### <a id='Overview'>1: Overview</a>

#### Motivation
To have **an interactive visual tour of** the change in estimated population in the US county level in the past 20 years through animated visulization.  The interactive visualization can be seen in my Tableau project [here](https://public.tableau.com/app/profile/kam.leung.yeung/viz/USCountyPopulationEstimateChangeBetween2000and2019/Dashboard1).  Below is a screenshot of the final presentation.


<a href="https://public.tableau.com/app/profile/kam.leung.yeung/viz/USCountyPopulationEstimateChangeBetween2000and2019/Dashboard1"><img src="Tableau_page.PNG" alt="Drawing" style="width: 800px;"/></a>

#### Method
Data from United States Census Bureau was acquired by downloading the related csv/excel files. The initial idea of acquiring data from [Population Estimates APIs](https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html) is not feasible due to the non-consecutive yearly estimate of the population at the county level.  

Data was prepared and summarized in pandas, and the exported data was fed to Tableau for creating the interactive visuals.



#### Author 
Kam Leung YEUNG  
samsam.yeung@gmail.com  
  
    

#### <a id='TOB'>Table of content</a>
<a href='#Overview'>1: Overview </a>  
<a href='#Section2'>2: Identifying and downloading the county-level poulation estimate datasets </a>  
<a href='#Section3'>3: Preparation of the 10-19 dataset</a>  
<a href='#Section4'>4: Preparation of the 00-09 dataset</a>  
<a href='#Section5'>5: Merging the two dataframes</a>  
<a href='#Section6'>6: Compute year-to-year change, melt df, export as csv</a>  


  
  
**---------------------------------------------------------------------------------------------------------------------**
  
  

### <a id='Section2'>2: Identifying and downloading the county-level poulation estimate datasets</a>

The main page of the US population estimate is [here](https://www.census.gov/programs-surveys/popest.html).  For the purpose of this project, I accessed the [2019 page](https://www.census.gov/programs-surveys/popest/data/tables.2019.List_58029271.html) and chose the [**County Population Totals: 2010-2019**](https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html).  The excel table for the US can be downloaded by clicking [United States](https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/co-est2019-annres.xlsx)

For the 2000-2010 dataset, it is hosted under this [page](https://www.census.gov/data/datasets/time-series/demo/popest/intercensal-2000-2010-counties.html), and the **csv file** can be downloaded with this [link](http://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/county/co-est00int-tot.csv)

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

<a href='#TOB'>Back to table of content</a>  

### <a id='Section3'>3: Preparation of the 10-19 dataset</a>

#### 3.1 Import data, a quick look

In [2]:
df10_19 = pd.read_excel('co-est2019-annres.xlsx',sheet_name='CO-EST2019-ANNRES', header =3)


In [3]:
df10_19.head(2) # row 0; col 'census', 'Estimates Base'

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.0,308758105.0,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
1,".Autauga County, Alabama",54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0


In [4]:
# which rows are non-data row? 3143-3148
df10_19.tail(7) 

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
3142,".Weston County, Wyoming",7208.0,7208.0,7198.0,7142.0,7077.0,7136.0,7138.0,7208.0,7220.0,6968.0,6924.0,6927.0
3143,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3144,"Note: The 6,222 people in Bedford city, Virgin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3145,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3146,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3147,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,Release Date: March 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.2 Give descriptive col name, drop non-data rows

In [5]:
# rename columns for clarify. other tables will have similar col names.
df10_19 = df10_19.rename( columns={"Census" : "Census_2010", "Estimates Base" : "Est_Base_2010"} )


In [6]:
# row 3143 - 3148 are not part of the data table. 
# delete those rows by indices
df10_19 = df10_19.drop(labels=0, axis=0)
df10_19 = df10_19.drop(labels=range(3143,3149), axis=0)
df10_19.head(3)

,Unnamed: 0,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,".Autauga County, Alabama",54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0
2,".Baldwin County, Alabama",182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0
3,".Barbour County, Alabama",27457.0,27455.0,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0


In [7]:
df10_19.tail(5)

,Unnamed: 0,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
3138,".Sweetwater County, Wyoming",43806.0,43806.0,43574.0,43986.0,45002.0,45157.0,44948.0,44719.0,44222.0,43464.0,42858.0,42343.0
3139,".Teton County, Wyoming",21294.0,21298.0,21296.0,21414.0,21624.0,22315.0,22773.0,23047.0,23234.0,23384.0,23269.0,23464.0
3140,".Uinta County, Wyoming",21118.0,21121.0,21089.0,20896.0,20996.0,20951.0,20822.0,20763.0,20682.0,20431.0,20292.0,20226.0
3141,".Washakie County, Wyoming",8533.0,8528.0,8530.0,8449.0,8409.0,8413.0,8273.0,8278.0,8165.0,8010.0,7877.0,7805.0
3142,".Weston County, Wyoming",7208.0,7208.0,7198.0,7142.0,7077.0,7136.0,7138.0,7208.0,7220.0,6968.0,6924.0,6927.0


In [8]:
# col "Unnamed: 0" is not informative of the info
# separate it into county,state cols
temp_df = df10_19["Unnamed: 0"].str.split(",", n = 1, expand = True)
temp_df.head(5)

,0,1
1,.Autauga County,Alabama
2,.Baldwin County,Alabama
3,.Barbour County,Alabama
4,.Bibb County,Alabama
5,.Blount County,Alabama


In [9]:
# keep all but the last word 
# ISSUE: 
#temp_df[0]=temp_df[0].str.split('\s+').str[:-1].apply(lambda parts: " ".join(parts))


#### 3.3 Col fixes, split col into state and county

In [10]:
# Problem found: "." in front of some county name
# remove the leading "." 
temp_df[0] = temp_df[0].str.replace(r'\.', '')

<ipython-input-10-608d4e2733ee>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  temp_df[0] = temp_df[0].str.replace(r'\.', '')


In [11]:
temp_df.head(3) # "." issue solved

,0,1
1,Autauga County,Alabama
2,Baldwin County,Alabama
3,Barbour County,Alabama


In [12]:
# add the col county and state to the main df from temp_df
df10_19['county'] = temp_df[0] # add the county column
df10_19['state'] = temp_df[1] # add the state column
df10_19 = df10_19.drop(["Unnamed: 0"], axis=1)# drop the now redundant column
df10_19.head(2)

,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,county,state
1,54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0,Autauga County,Alabama
2,182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0,Baldwin County,Alabama


In [13]:
# reorder the column for easier read
df10_19.insert(0, 'County', df10_19['county'])
df10_19.insert(1, 'State', df10_19['state'])
df10_19 = df10_19.drop(['county','state'], axis=1)
df10_19.head(3) # looking good

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Autauga County,Alabama,54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0
2,Baldwin County,Alabama,182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0
3,Barbour County,Alabama,27457.0,27455.0,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0


<a href='#TOB'>Back to table of content</a>  

### <a id='Section4'>4: Preparation of the 00-09 dataset</a>

#### 4.1 Import data, quick inspection

In [14]:
df00_09 = pd.read_excel('co-est00int-tot.xlsx',sheet_name='co-est00int-tot')
df00_09.head(3)
# the 00_09 dataset also has: CENSUS2010POP, POPESTIMATE2010, so drop them

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,ESTIMATESBASE2000,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP,POPESTIMATE2010
0,40,3,6,1,0,Alabama,Alabama,4447207,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938,4779736,4785298
1,50,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,54571,54632
2,50,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,182265,183195


In [15]:
df00_09.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3194 entries, 0 to 3193
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SUMLEV             3194 non-null   int64 
 1   REGION             3194 non-null   int64 
 2   DIVISION           3194 non-null   int64 
 3   STATE              3194 non-null   int64 
 4   COUNTY             3194 non-null   int64 
 5   STNAME             3194 non-null   object
 6   CTYNAME            3194 non-null   object
 7   ESTIMATESBASE2000  3194 non-null   int64 
 8   POPESTIMATE2000    3194 non-null   int64 
 9   POPESTIMATE2001    3194 non-null   int64 
 10  POPESTIMATE2002    3194 non-null   int64 
 11  POPESTIMATE2003    3194 non-null   int64 
 12  POPESTIMATE2004    3194 non-null   int64 
 13  POPESTIMATE2005    3194 non-null   int64 
 14  POPESTIMATE2006    3194 non-null   int64 
 15  POPESTIMATE2007    3194 non-null   int64 
 16  POPESTIMATE2008    3194 non-null   int64 


#### 4.2 Give descriptive col names, drop repeated cols

In [16]:
# drop rows where county and state names are the same: SUMLEV =40

# drop SUMLEV, CENSUS2010POP POPESTIMATE2010 (both in previous file)
df00_09.drop(['CENSUS2010POP','POPESTIMATE2010'], axis=1,inplace=True)

# rename STNAME-->State, CTYNAME-->County, ESTIMATESBASE2000-->Est_Base_2000
df00_09= df00_09.rename(columns={"STNAME": "State", "CTYNAME": "County", "ESTIMATESBASE2000": "Est_Base_2000"})

# rename: POPESTIMATE200X-->200X (2000-2009)
df00_09= df00_09.rename(columns={"POPESTIMATE2000": "2000", "POPESTIMATE2001": "2001", "POPESTIMATE2002": "2002"})
df00_09= df00_09.rename(columns={"POPESTIMATE2003": "2003", "POPESTIMATE2004": "2004", "POPESTIMATE2005": "2005"})
df00_09= df00_09.rename(columns={"POPESTIMATE2006": "2006", "POPESTIMATE2007": "2007", "POPESTIMATE2008": "2008"})
df00_09= df00_09.rename(columns={"POPESTIMATE2009": "2009"})

df00_09.head(3)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,40,3,6,1,0,Alabama,Alabama,4447207,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938
1,50,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135
2,50,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406


In [17]:
# check state-level rows
df = df00_09[df00_09['SUMLEV'] == 40]
df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,40,3,6,1,0,Alabama,Alabama,4447207,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938
68,40,4,9,2,0,Alaska,Alaska,626933,627963,633714,642337,648414,659286,666946,675302,680300,687455,698895
98,40,4,8,4,0,Arizona,Arizona,5130247,5160586,5273477,5396255,5510364,5652404,5839077,6029141,6167681,6280362,6343154
114,40,3,7,5,0,Arkansas,Arkansas,2673293,2678588,2691571,2705927,2724816,2749686,2781097,2821761,2848650,2874554,2896843
190,40,4,9,6,0,California,California,33871653,33987977,34479458,34871843,35253159,35574576,35827943,36021202,36250311,36604337,36961229
249,40,4,8,8,0,Colorado,Colorado,4302086,4326921,4425687,4490406,4528732,4575013,4631888,4720423,4803868,4889730,4972195
314,40,1,1,9,0,Connecticut,Connecticut,3405650,3411777,3432835,3458749,3484336,3496094,3506956,3517460,3527270,3545579,3561807
323,40,3,5,10,0,Delaware,Delaware,783559,786373,795699,806169,818003,830803,845150,859268,871749,883874,891730
327,40,3,5,11,0,District of Columbia,District of Columbia,572086,572046,574504,573158,568502,567754,567136,570681,574404,580236,592228
329,40,3,5,12,0,Florida,Florida,15982571,16047515,16356966,16689370,17004085,17415318,17842038,18166990,18367842,18527305,18652644


In [18]:
len(df)

51

#### 4.3 Drop rows with total state pop (SUMLEV=40); drop col SUMLEV

In [19]:
len(df00_09)

3194

In [20]:
# remove row that has total state pop (SUMLEV==40)
df00_09 = df00_09[df00_09['SUMLEV'] != 40]
len(df00_09)

3143

In [21]:
# SUMLEV no longer needed, drop it
# drop SUMLEV, CENSUS2010POP POPESTIMATE2010 (both in previous file)
df00_09.drop(['SUMLEV'], axis=1,inplace=True)

In [22]:
df00_09.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 1 to 3193
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   REGION         3143 non-null   int64 
 1   DIVISION       3143 non-null   int64 
 2   STATE          3143 non-null   int64 
 3   COUNTY         3143 non-null   int64 
 4   State          3143 non-null   object
 5   County         3143 non-null   object
 6   Est_Base_2000  3143 non-null   int64 
 7   2000           3143 non-null   int64 
 8   2001           3143 non-null   int64 
 9   2002           3143 non-null   int64 
 10  2003           3143 non-null   int64 
 11  2004           3143 non-null   int64 
 12  2005           3143 non-null   int64 
 13  2006           3143 non-null   int64 
 14  2007           3143 non-null   int64 
 15  2008           3143 non-null   int64 
 16  2009           3143 non-null   int64 
dtypes: int64(15), object(2)
memory usage: 442.0+ KB


#### 4.4 Change to appropriate data type and create the col FIPS

In [23]:
# change the dtype to str before padding leading zeros
df00_09['STATE'] = df00_09['STATE'].astype(str)
df00_09['COUNTY'] = df00_09['COUNTY'].astype(str)
# create FIPS from state and county
df00_09['FIPS'] = df00_09['STATE'].str.zfill(2)+df00_09['COUNTY'].str.zfill(3)#+str(df00_09['COUNTY']).zfill(3) 
df00_09[{'2009','FIPS'}].head(5)

,2009,FIPS
1,54135,01001
2,179406,01003
3,27657,01005
4,22941,01007
5,57341,01009


In [24]:
df00_09.head(3)

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
1,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,01001
2,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,01003
3,3,6,1,5,Alabama,Barbour County,29042,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,01005


<a href='#TOB'>Back to table of content</a>  

### <a id='Section5'>5: Merging the two dataframes</a>

#### 5.1 Initial merge attempt and issues discovery 

In [25]:
len(df00_09)

3143

In [26]:
len(df10_19)

3142

In [27]:
combined = df00_09.merge(df10_19, on=['State','County'], how='left')
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 0 to 3142
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   REGION         3143 non-null   int64  
 1   DIVISION       3143 non-null   int64  
 2   STATE          3143 non-null   object 
 3   COUNTY         3143 non-null   object 
 4   State          3143 non-null   object 
 5   County         3143 non-null   object 
 6   Est_Base_2000  3143 non-null   int64  
 7   2000           3143 non-null   int64  
 8   2001           3143 non-null   int64  
 9   2002           3143 non-null   int64  
 10  2003           3143 non-null   int64  
 11  2004           3143 non-null   int64  
 12  2005           3143 non-null   int64  
 13  2006           3143 non-null   int64  
 14  2007           3143 non-null   int64  
 15  2008           3143 non-null   int64  
 16  2009           3143 non-null   int64  
 17  FIPS           3143 non-null   object 
 18  Census_2

Zero row from df10_19 is merged

In [28]:
combined.head(3)

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,01001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,01003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6,1,5,Alabama,Barbour County,29042,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,01005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 5.2 Merge fix: unsuccessful keys match (state, county)

 unsuccessful merge on two keys: state and county.  Find out what's wrong

In [29]:
df10_19.head(3)

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Autauga County,Alabama,54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0
2,Baldwin County,Alabama,182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0
3,Barbour County,Alabama,27457.0,27455.0,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0


In [30]:
df00_09.head(3)

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
1,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,01001
2,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,01003
3,3,6,1,5,Alabama,Barbour County,29042,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,01005


In [31]:
df00_09['State'][1]

'Alabama'

In [32]:
df10_19['State'][1] 

' Alabama'

Extra space found in state col in df10_19

In [33]:
# remove leading space in state name in df10_19
df10_19['State']=df10_19['State'].str.strip()

In [34]:
df10_19['County'][1]

'Autauga County'

#### 5.3 Merge attempt 2

In [35]:
combined = df00_09.merge(df10_19, on=['State','County'], how='left')
combined.head(10)

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,01001,54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0
1,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,01003,182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0
2,3,6,1,5,Alabama,Barbour County,29042,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,01005,27457.0,27455.0,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0
3,3,6,1,7,Alabama,Bibb County,19856,19913,21028,21199,21399,21721,22042,22099,22438,22705,22941,01007,22915.0,22915.0,22870.0,22745.0,22667.0,22521.0,22553.0,22566.0,22586.0,22550.0,22367.0,22394.0
4,3,6,1,9,Alabama,Blount County,50982,51107,51845,52551,53457,54124,54624,55485,56240,57055,57341,01009,57322.0,57322.0,57376.0,57560.0,57580.0,57619.0,57526.0,57526.0,57494.0,57787.0,57771.0,57826.0
5,3,6,1,11,Alabama,Bullock County,11603,11581,11358,11256,11316,11056,11011,10776,11011,10953,10987,01011,10914.0,10911.0,10876.0,10675.0,10606.0,10549.0,10663.0,10400.0,10389.0,10176.0,10174.0,10101.0
6,3,6,1,13,Alabama,Butler County,21394,21325,21139,20803,20833,20870,20830,20815,20894,20949,20867,01013,20947.0,20940.0,20932.0,20866.0,20670.0,20356.0,20327.0,20162.0,20012.0,19888.0,19631.0,19448.0
7,3,6,1,15,Alabama,Calhoun County,111882,111081,111266,111625,112705,113462,114477,115388,116211,117274,118363,01015,118572.0,118526.0,118408.0,117744.0,117190.0,116471.0,115917.0,115469.0,114973.0,114710.0,114331.0,113605.0
8,3,6,1,17,Alabama,Chambers County,36600,36571,36274,35965,35680,35463,35279,34945,34847,34563,34384,01017,34215.0,34169.0,34122.0,34033.0,34104.0,34139.0,33977.0,33996.0,33745.0,33707.0,33600.0,33254.0
9,3,6,1,19,Alabama,Cherokee County,23909,24006,24182,24403,24591,24887,25031,25466,25553,25636,25854,01019,25989.0,25979.0,25963.0,25989.0,25958.0,26017.0,25895.0,25732.0,25768.0,25805.0,26014.0,26196.0


In [36]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 0 to 3142
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   REGION         3143 non-null   int64  
 1   DIVISION       3143 non-null   int64  
 2   STATE          3143 non-null   object 
 3   COUNTY         3143 non-null   object 
 4   State          3143 non-null   object 
 5   County         3143 non-null   object 
 6   Est_Base_2000  3143 non-null   int64  
 7   2000           3143 non-null   int64  
 8   2001           3143 non-null   int64  
 9   2002           3143 non-null   int64  
 10  2003           3143 non-null   int64  
 11  2004           3143 non-null   int64  
 12  2005           3143 non-null   int64  
 13  2006           3143 non-null   int64  
 14  2007           3143 non-null   int64  
 15  2008           3143 non-null   int64  
 16  2009           3143 non-null   int64  
 17  FIPS           3143 non-null   object 
 18  Census_2

Only 3111 rows from df10_19 are merged

#### 5.4 Found out why only 3111 rows from df10_19 is matched to df00_09's 3143

In [37]:
# show rows that cannot find a match in combined
bool_series = pd.isnull(combined['Census_2010'])
combined[bool_series]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
57,3,6,1,115,Alabama,St. Clair County,64740,65080,66093,67274,68629,70416,72596,75948,79348,81223,83009,01115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,4,9,2,195,Alaska,Petersburg Census Area,4256,4249,4198,4112,4022,4059,3959,3886,3898,3817,3775,02195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,4,9,2,270,Alaska,Wade Hampton Census Area,7028,7033,7051,7103,7201,7330,7313,7292,7372,7392,7469,02270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,3,7,5,123,Arkansas,St. Francis County,29330,29325,29072,29023,29170,28905,28639,28496,28394,28110,28336,05123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,3,5,12,109,Florida,St. Johns County,123237,124522,129805,135770,142165,151717,160266,168480,175318,181720,186281,12109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,3,5,12,111,Florida,St. Lucie County,192679,193518,198459,205796,214924,228548,241965,255251,266860,273151,274344,12111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,2,3,17,163,Illinois,St. Clair County,256043,256462,256540,257952,258448,260214,261255,263445,264764,266518,268489,17163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767,2,3,18,141,Indiana,St. Joseph County,265567,265845,265502,265090,264762,264541,264863,265496,266045,267282,266827,18141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,3,7,22,59,Louisiana,La Salle Parish,14276,14271,14207,14359,14356,14366,14313,14519,14570,14667,14717,22059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,3,7,22,87,Louisiana,St. Bernard Parish,67230,67278,68028,68964,69621,70547,71300,16563,23613,28879,32878,22087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Is there issue in the county name?
df00_09[df00_09['County'].str.contains('Clair')]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
58,3,6,1,115,Alabama,St. Clair County,64740,65080,66093,67274,68629,70416,72596,75948,79348,81223,83009,01115
690,2,3,17,163,Illinois,St. Clair County,256043,256462,256540,257952,258448,260214,261255,263445,264764,266518,268489,17163
1327,2,3,26,147,Michigan,St. Clair County,164242,164621,165444,166086,167549,168457,168209,168312,167526,165959,164011,26147
1601,2,4,29,185,Missouri,St. Clair County,9652,9698,9656,9692,9656,9711,9733,9694,9802,9757,9708,29185
3115,2,3,55,35,Wisconsin,Eau Claire County,93128,93310,93835,94493,94632,94618,95007,95896,96632,97176,98274,55035


In [39]:
df10_19[df10_19['County'].str.contains('Clair')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
58,St Clair County,Alabama,83593.0,83350.0,83571.0,83975.0,84761.0,85810.0,85996.0,86546.0,87306.0,87926.0,88654.0,89512.0
677,St Clair County,Illinois,270056.0,270078.0,270353.0,270048.0,268673.0,266831.0,265941.0,264858.0,263187.0,262600.0,261360.0,259686.0
1305,St Clair County,Michigan,163040.0,163051.0,162690.0,161581.0,160625.0,160288.0,160065.0,159700.0,159369.0,159005.0,159031.0,159128.0
1576,St Clair County,Missouri,9805.0,9805.0,9823.0,9693.0,9527.0,9487.0,9432.0,9411.0,9288.0,9365.0,9389.0,9397.0
3065,Eau Claire County,Wisconsin,98736.0,98881.0,99017.0,99938.0,100839.0,101734.0,101690.0,102080.0,102907.0,103582.0,104355.0,104646.0


An extra period (.) in df00_09 when abbreviation is used (e.g., St. Clair County)

In [40]:
# df10_19 county name "St Clair" does not use . as in "St. Clair"
# remove the "." in df00_09
df00_09['County'] = df00_09['County'].str.replace(r'\.', '')
df00_09[df00_09['County'].str.contains('Clair')]

<ipython-input-40-095310dc10c2>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df00_09['County'] = df00_09['County'].str.replace(r'\.', '')


,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
58,3,6,1,115,Alabama,St Clair County,64740,65080,66093,67274,68629,70416,72596,75948,79348,81223,83009,01115
690,2,3,17,163,Illinois,St Clair County,256043,256462,256540,257952,258448,260214,261255,263445,264764,266518,268489,17163
1327,2,3,26,147,Michigan,St Clair County,164242,164621,165444,166086,167549,168457,168209,168312,167526,165959,164011,26147
1601,2,4,29,185,Missouri,St Clair County,9652,9698,9656,9692,9656,9711,9733,9694,9802,9757,9708,29185
3115,2,3,55,35,Wisconsin,Eau Claire County,93128,93310,93835,94493,94632,94618,95007,95896,96632,97176,98274,55035


#### 5.5 Merge attempt 3 and more fixes

In [41]:
# merge again and find more issues
combined = df00_09.merge(df10_19, on=['State','County'], how='left')
bool_series = pd.isnull(combined['Census_2010'])
combined[bool_series]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
86,4,9,2,195,Alaska,Petersburg Census Area,4256,4249,4198,4112,4022,4059,3959,3886,3898,3817,3775,02195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,4,9,2,270,Alaska,Wade Hampton Census Area,7028,7033,7051,7103,7201,7330,7313,7292,7372,7392,7469,02270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,3,7,22,59,Louisiana,La Salle Parish,14276,14271,14207,14359,14356,14366,14313,14519,14570,14667,14717,22059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2417,2,4,46,113,South Dakota,Shannon County,12466,12542,12602,12872,12993,12983,13150,13404,13345,13368,13425,46113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2916,3,5,51,515,Virginia,Bedford city,6421,6425,6400,6320,6237,6198,6186,6147,6169,6150,6174,51515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


5 rows in df10_19 still cannot find its match in the merge

In [42]:
df10_19[df10_19['County'].str.contains('Salle') & df10_19['State'].str.contains('Louisiana')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1143,LaSalle Parish,Louisiana,14890.0,14890.0,14908.0,14941.0,14862.0,14821.0,14875.0,14979.0,15022.0,14887.0,14901.0,14892.0


No space in "La Salle Parish" in df10_19

In [43]:
# remove space in "La Salle" in df00_09
filter1=df00_09[df00_09['County'].str.contains('La Salle Parish') & df00_09['State'].str.contains('Louisiana')]\
.index.values.tolist()
df00_09.loc[filter1,'County']= "LaSalle Parish"

In [44]:
# check if change is as intended
df00_09[df00_09['County'].str.contains('LaSalle Parish') & df00_09['State'].str.contains('Louisiana')]['County']

1161    LaSalle Parish
Name: County, dtype: object

In [45]:
df10_19[df10_19['County'].str.contains('Shannon') & df10_19['State'].str.contains('South Dakota')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019


##### [Here](https://www.ddorn.net/data/FIPS_County_Code_Changes.pdf) shows that   
"South Dakota, 2015: Shannon County (FIPS 46113) is renamed to Oglala Lakota County (FIPS 46102) as of May 1st, 2015.  
Action: replace FIPS code 46102 with the old code 46113."

In [46]:
df10_19[df10_19['County'].str.contains('Oglala') & df10_19['State'].str.contains('South Dakota')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2413,Oglala Lakota County,South Dakota,13586.0,13586.0,13636.0,13896.0,14037.0,14126.0,14213.0,14358.0,14425.0,14384.0,14331.0,14177.0


In [47]:
# change df00_09 county's name to Oglala Lakota County
# also change COUNTY, and FIPS to updated values
filter2=df00_09[df00_09['County'].str.contains('Shannon') & df00_09['State'].str.contains('South Dakota')]\
.index.values.tolist()
df00_09.loc[filter2,'County']= "Oglala Lakota County"
df00_09.loc[filter2,'COUNTY']= "102"
df00_09.loc[filter2,'FIPS']= "46102"

In [48]:
df00_09[df00_09['County'].str.contains('Oglala Lakota County') & df00_09['State'].str.contains('South Dakota')]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
2459,2,4,46,102,South Dakota,Oglala Lakota County,12466,12542,12602,12872,12993,12983,13150,13404,13345,13368,13425,46102


#### 5.6 Merge attempt 4 and more fixes

In [49]:
# merge again and find more issues
combined = df00_09.merge(df10_19, on=['State','County'], how='left')
bool_series = pd.isnull(combined['Census_2010'])
combined[bool_series]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
86,4,9,2,195,Alaska,Petersburg Census Area,4256,4249,4198,4112,4022,4059,3959,3886,3898,3817,3775,02195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,4,9,2,270,Alaska,Wade Hampton Census Area,7028,7033,7051,7103,7201,7330,7313,7292,7372,7392,7469,02270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2916,3,5,51,515,Virginia,Bedford city,6421,6425,6400,6320,6237,6198,6186,6147,6169,6150,6174,51515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3 rows in df10_19 cannot find their match in the merge

In [50]:
df10_19[df10_19['County'].str.contains('Petersburg') & df10_19['State'].str.contains('Alaska')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
88,Petersburg Borough,Alaska,3815.0,3207.0,3219.0,3255.0,3277.0,3291.0,3261.0,3253.0,3260.0,3264.0,3244.0,3266.0


##### Exclude Petersburg, Alaska
**Reason:** 
1. Great difference on pop between 2009 and 2010: df00_09 uses "Petersburg Census Area" (2009 pop 3775) while df10_19 uses "Petersburg Borough" (2010 pop: 3219) 
2. No info can be found to reconcile the diff between "Petersburg Census Area" and "Petersburg Borough"

In [51]:
# double check to get the correct index for deletion
df00_09[df00_09['County'].str.contains('Petersburg') & df00_09['State'].str.contains('Alaska')]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
88,4,9,2,195,Alaska,Petersburg Census Area,4256,4249,4198,4112,4022,4059,3959,3886,3898,3817,3775,02195


In [52]:
df00_09 = df00_09.drop(labels=88,axis=0)

In [53]:
combined[bool_series]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
86,4,9,2,195,Alaska,Petersburg Census Area,4256,4249,4198,4112,4022,4059,3959,3886,3898,3817,3775,02195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,4,9,2,270,Alaska,Wade Hampton Census Area,7028,7033,7051,7103,7201,7330,7313,7292,7372,7392,7469,02270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2916,3,5,51,515,Virginia,Bedford city,6421,6425,6400,6320,6237,6198,6186,6147,6169,6150,6174,51515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df00_09[df00_09['County'].str.contains('Bedford') & df00_09['State'].str.contains('Virginia')]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
2876,3,5,51,19,Virginia,Bedford County,60226,60472,60985,61580,62441,63563,64825,66363,67392,67594,68399,51019
2963,3,5,51,515,Virginia,Bedford city,6421,6425,6400,6320,6237,6198,6186,6147,6169,6150,6174,51515


In [55]:
df10_19[df10_19['County'].str.contains('Bedford') & df10_19['State'].str.contains('Virginia')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2830,Bedford County,Virginia,68676.0,74929.0,75035.0,75554.0,75750.0,76177.0,77034.0,77629.0,77993.0,78378.0,78882.0,78997.0


##### [Here](https://www.ddorn.net/data/FIPS_County_Code_Changes.pdf) shows that   
"Virginia, 2013: Bedford (independent) city (FIPS 51515) was changed to town status and added to Bedford County (FIPS 51019) effective July 1st, 2013.  
Action: no adjustment needed since Bedford (independent) city and Bedford County map into the same CZ 2300."  
  
**Solution**: In df00_09
1. add Bedford city's estimate to Bedford County
2. delete bedford city, virginia 

In [56]:
col_names = list(df00_09.columns.values[6:-1])

In [57]:
df00_09.loc[2876,col_names] = df00_09.loc[2876,col_names] + df00_09.loc[2963,col_names]

In [58]:
df00_09[df00_09['County'].str.contains('Bedford') & df00_09['State'].str.contains('Virginia')]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
2876,3,5,51,19,Virginia,Bedford County,66647,66897,67385,67900,68678,69761,71011,72510,73561,73744,74573,51019
2963,3,5,51,515,Virginia,Bedford city,6421,6425,6400,6320,6237,6198,6186,6147,6169,6150,6174,51515


In [59]:
# delete the row with Bedford city
df00_09 = df00_09.drop(labels=2963, axis=0)

#### 5.7 Merge attempt 6 and final fix

[Wiki](https://en.wikipedia.org/wiki/Kusilvak_Census_Area,_Alaska) shows that "Kusilvak Census Area, formerly known as Wade Hampton Census Area"  
  
[Here](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwi56bHRmcjyAhVYZ80KHZKQDa4QFnoECAMQAQ&url=https%3A%2F%2Fwww.cdc.gov%2Fnchs%2Fnvss%2Fbridged_race%2Fcounty_geography-_changes2015.pdf&usg=AOvVaw2VLX4359bRP-r0dvb3xCLX) shows that "Kusilvak Census Area, AK (FIPS code=02158)"

**Solution**
1. In df00_09, change COUNTY (to 158), FIPS (to 02158), and County (to Kusilvak Census Area)

In [60]:
# merge again and find more issues
combined = df00_09.merge(df10_19, on=['State','County'], how='left')
bool_series = pd.isnull(combined['Census_2010'])
combined[bool_series]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
91,4,9,2,270,Alaska,Wade Hampton Census Area,7028,7033,7051,7103,7201,7330,7313,7292,7372,7392,7469,02270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


One last row in df10_19 that cannot find its match

In [61]:
df10_19[df10_19['County'].str.contains('Kusilvak') & df10_19['State'].str.contains('Alaska')]

,County,State,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
82,Kusilvak Census Area,Alaska,7459.0,7459.0,7467.0,7651.0,7794.0,7974.0,8100.0,8194.0,8181.0,8236.0,8323.0,8314.0


In [62]:
filter3=df00_09[df00_09['County'].str.contains('Wade Hampton Census Area') & df00_09['State'].str.contains('Alaska')]\
.index.values.tolist()
df00_09.loc[filter3,'County']= "Kusilvak Census Area"
df00_09.loc[filter3,'COUNTY']= "102"
df00_09.loc[filter3,'FIPS']= "02158"

In [63]:
# check if the change is effective
df00_09[df00_09['County'].str.contains('Kusilvak Census Area') & df00_09['State'].str.contains('Alaska')]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
94,4,9,2,102,Alaska,Kusilvak Census Area,7028,7033,7051,7103,7201,7330,7313,7292,7372,7392,7469,02158


In [64]:
# merge again and find more issues
combined = df00_09.merge(df10_19, on=['State','County'], how='left')
bool_series = pd.isnull(combined['Census_2010'])
combined[bool_series]

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019


In [65]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3141 entries, 0 to 3140
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   REGION         3141 non-null   int64  
 1   DIVISION       3141 non-null   int64  
 2   STATE          3141 non-null   object 
 3   COUNTY         3141 non-null   object 
 4   State          3141 non-null   object 
 5   County         3141 non-null   object 
 6   Est_Base_2000  3141 non-null   int64  
 7   2000           3141 non-null   int64  
 8   2001           3141 non-null   int64  
 9   2002           3141 non-null   int64  
 10  2003           3141 non-null   int64  
 11  2004           3141 non-null   int64  
 12  2005           3141 non-null   int64  
 13  2006           3141 non-null   int64  
 14  2007           3141 non-null   int64  
 15  2008           3141 non-null   int64  
 16  2009           3141 non-null   int64  
 17  FIPS           3141 non-null   object 
 18  Census_2

Good! All columns have 3141 non-null rows 

In [66]:
combined.head(3)

,REGION,DIVISION,STATE,COUNTY,State,County,Est_Base_2000,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS,Census_2010,Est_Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,3,6,1,1,Alabama,Autauga County,43751,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,01001,54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0
1,3,6,1,3,Alabama,Baldwin County,140416,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,01003,182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0
2,3,6,1,5,Alabama,Barbour County,29042,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,01005,27457.0,27455.0,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0


<a href='#TOB'>Back to table of content</a>  

### <a id='Section6'>6: Compute year-to-year change, melt df, export as csv</a>

#### 6.1 compute year-to-year change, in % and diff on the combined df

In [67]:
# re-order col for ease of reading
names = list(combined.columns.values)
names

['REGION',
 'DIVISION',
 'STATE',
 'COUNTY',
 'State',
 'County',
 'Est_Base_2000',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 'FIPS',
 'Census_2010',
 'Est_Base_2010',
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [68]:
combined=combined[[
'REGION',
'DIVISION',
'STATE',
'COUNTY',
'FIPS',
'State',
'County',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
    2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
    'Est_Base_2000',
     'Census_2010',
 'Est_Base_2010',
    
]]
combined.head(3)

,REGION,DIVISION,STATE,COUNTY,FIPS,State,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Est_Base_2000,Census_2010,Est_Base_2010
0,3,6,1,1,01001,Alabama,Autauga County,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0,43751,54571.0,54597.0
1,3,6,1,3,01003,Alabama,Baldwin County,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0,140416,182265.0,182265.0
2,3,6,1,5,01005,Alabama,Barbour County,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0,29042,27457.0,27455.0


In [69]:
# get the col names
col_name = list(combined.columns.values)

In [70]:
# compute the year to year change
for i in range(8,27,1):
    # print(i)
    per_name = str(combined.columns[i])+"_per_change"
    diff_name = str(combined.columns[i])+"_diff_change"
    cul_per_name = str(combined.columns[i])+"cul_per_change"
    cul_diff_name = str(combined.columns[i])+"cul_diff_change"
    
    #print(col_name[i])
    #print(col_name[(i-1)])
    #print("\n")
    combined[per_name] = (combined[col_name[i]]-combined[col_name[(i-1)]])/combined[col_name[(i-1)]]
    combined[diff_name] = (combined[col_name[i]]-combined[col_name[(i-1)]])
    combined[cul_per_name] = (combined[col_name[i]]-combined[col_name[7]])/combined[col_name[7]]
    combined[cul_diff_name] = (combined[col_name[i]]-combined[col_name[7]])

combined.head(100)

,REGION,DIVISION,STATE,COUNTY,FIPS,State,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Est_Base_2000,Census_2010,Est_Base_2010,2001_per_change,2001_diff_change,2001cul_per_change,2001cul_diff_change,2002_per_change,2002_diff_change,2002cul_per_change,2002cul_diff_change,2003_per_change,2003_diff_change,2003cul_per_change,2003cul_diff_change,2004_per_change,2004_diff_change,2004cul_per_change,2004cul_diff_change,2005_per_change,2005_diff_change,2005cul_per_change,2005cul_diff_change,2006_per_change,2006_diff_change,2006cul_per_change,2006cul_diff_change,2007_per_change,2007_diff_change,2007cul_per_change,2007cul_diff_change,2008_per_change,2008_diff_change,2008cul_per_change,2008cul_diff_change,2009_per_change,2009_diff_change,2009cul_per_change,2009cul_diff_change,2010_per_change,2010_diff_change,2010cul_per_change,2010cul_diff_change,2011_per_change,2011_diff_change,2011cul_per_change,2011cul_diff_change,2012_per_change,2012_diff_change,2012cul_per_change,2012cul_diff_change,2013_per_change,2013_diff_change,2013cul_per_change,2013cul_diff_change,2014_per_change,2014_diff_change,2014cul_per_change,2014cul_diff_change,2015_per_change,2015_diff_change,2015cul_per_change,2015cul_diff_change,2016_per_change,2016_diff_change,2016cul_per_change,2016cul_diff_change,2017_per_change,2017_diff_change,2017cul_per_change,2017cul_diff_change,2018_per_change,2018_diff_change,2018cul_per_change,2018cul_diff_change,2019_per_change,2019_diff_change,2019cul_per_change,2019cul_diff_change
0,3,6,1,1,01001,Alabama,Autauga County,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0,43751,54571.0,54597.0,0.019718,868,0.019718,868,0.022723,1020,0.042889,1888,0.019408,891,0.063129,2779,0.033462,1566,0.098703,4345,0.027085,1310,0.128461,5655,0.033255,1652,0.165989,7307,0.020983,1077,0.190455,8384,0.016640,872,0.210263,9256,0.016105,858,0.229754,10114,0.011785,638.0,0.244247,10752.0,0.008289,454.0,0.254560,11206.0,-0.004943,-273.0,0.248359,10933.0,-0.004131,-227.0,0.243202,10706.0,0.003033,166.0,0.246973,10872.0,-0.000528,-29.0,0.246314,10843.0,0.006908,379.0,0.254924,11222.0,0.002661,147.0,0.258263,11369.0,0.002582,143.0,0.261512,11512.0,0.006050,336.0,0.269144,11848.0
1,3,6,1,3,01003,Alabama,Baldwin County,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0,140416,182265.0,182265.0,0.024996,3533,0.024996,3533,0.021274,3082,0.046801,6615,0.024007,3552,0.071932,10167,0.031397,4757,0.105588,14924,0.037865,5917,0.147451,20841,0.036613,5938,0.189462,26779,0.025476,4283,0.219765,31062,0.019855,3423,0.243983,34485,0.020355,3579,0.269304,38064,0.020657,3706.0,0.295524,41770.0,0.018819,3446.0,0.319905,45216.0,0.019227,3587.0,0.345283,48803.0,0.024928,4740.0,0.378819,53543.0,0.022054,4298.0,0.409227,57841.0,0.018857,3756.0,0.435801,61597.0,0.022972,4662.0,0.468785,66259.0,0.023699,4920.0,0.503594,71179.0,0.025099,5334.0,0.541332,76513.0,0.024691,5379.0,0.579389,81892.0
2,3,6,1,5,01005,Alabama,Barbour County,29015,28863,28653,28594,28287,28027,27861,27757,27808,27657,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0,29042,27457.0,27455.0,-0.005239,-152,-0.005239,-152,-0.007276,-210,-0.012476,-362,-0.002059,-59,-0.014510,-421,-0.010737,-307,-0.025090,-728,-0.009192,-260,-0.034051,-988,-0.005923,-166,-0.039773,-1154,-0.003733,-104,-0.043357,-1258,0.001837,51,-0.041599,-1207,-0.005430,-151,-0.046803,-1358,-0.011932,-330.0,-0.058177,-1688.0,0.000512,14.0,-0.057694,-1674.0,-0.006291,-172.0,-0.063622,-1846.0,-0.008539,-232.0,-0.071618,-2078.0,-0.006757,-182.0,-0.077891,-2260.0,-0.017642,-472.0,-0.094158,-2732.0,-0.018149,-477.0,-0.110598,-3209.0,-0.025149,-649.0,-0.132966,-3858.0,-0.011329,-285.0,-0.142788,-4143.0,-0.007478,-186.0,-0.149199,-4329.0
3,3,6,1,7,01007,Alabama,Bibb County,199

#### 6.2 melt df for Tableau

In [71]:
# prepare the year names
year=[str(i) for i in range(2001,2020,1)]
year

['2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

In [72]:
# find the start and end index for difference in change
a=list(combined.columns.values)
a.index('2001_diff_change') # start

31

In [73]:
a.index('2019_diff_change') # + 1

103

In [74]:
diff_colname = list(combined.columns[31:104:4])
diff_colname

['2001_diff_change',
 '2002_diff_change',
 '2003_diff_change',
 '2004_diff_change',
 '2005_diff_change',
 '2006_diff_change',
 '2007_diff_change',
 '2008_diff_change',
 '2009_diff_change',
 '2010_diff_change',
 '2011_diff_change',
 '2012_diff_change',
 '2013_diff_change',
 '2014_diff_change',
 '2015_diff_change',
 '2016_diff_change',
 '2017_diff_change',
 '2018_diff_change',
 '2019_diff_change']

In [75]:
# find the start and end index for percentage in change
a.index('2001_per_change') # start

30

In [76]:
a.index('2019_per_change') # + 1

102

In [77]:
change_colname = list(combined.columns[30:103:4])
change_colname

['2001_per_change',
 '2002_per_change',
 '2003_per_change',
 '2004_per_change',
 '2005_per_change',
 '2006_per_change',
 '2007_per_change',
 '2008_per_change',
 '2009_per_change',
 '2010_per_change',
 '2011_per_change',
 '2012_per_change',
 '2013_per_change',
 '2014_per_change',
 '2015_per_change',
 '2016_per_change',
 '2017_per_change',
 '2018_per_change',
 '2019_per_change']

In [78]:
# find the start and end index for culminative percentage change since year 2000
a.index('2001cul_per_change') # start

32

In [79]:
a.index('2019cul_per_change') # + 1

104

In [80]:
cul_change_colname = list(combined.columns[32:105:4])
cul_change_colname

['2001cul_per_change',
 '2002cul_per_change',
 '2003cul_per_change',
 '2004cul_per_change',
 '2005cul_per_change',
 '2006cul_per_change',
 '2007cul_per_change',
 '2008cul_per_change',
 '2009cul_per_change',
 '2010cul_per_change',
 '2011cul_per_change',
 '2012cul_per_change',
 '2013cul_per_change',
 '2014cul_per_change',
 '2015cul_per_change',
 '2016cul_per_change',
 '2017cul_per_change',
 '2018cul_per_change',
 '2019cul_per_change']

In [81]:
# find the start and end index for culminative change since year 2000
a.index('2001cul_diff_change') # start

33

In [82]:
a.index('2019cul_diff_change') # + 1

105

In [83]:
cul_diff_colname = list(combined.columns[33:106:4])
cul_diff_colname

['2001cul_diff_change',
 '2002cul_diff_change',
 '2003cul_diff_change',
 '2004cul_diff_change',
 '2005cul_diff_change',
 '2006cul_diff_change',
 '2007cul_diff_change',
 '2008cul_diff_change',
 '2009cul_diff_change',
 '2010cul_diff_change',
 '2011cul_diff_change',
 '2012cul_diff_change',
 '2013cul_diff_change',
 '2014cul_diff_change',
 '2015cul_diff_change',
 '2016cul_diff_change',
 '2017cul_diff_change',
 '2018cul_diff_change',
 '2019cul_diff_change']

In [84]:
# find the start and end index for year 2001 and 2019
a.index('2001')

8

In [85]:
a.index(2019)

26

In [86]:
pop_address=[i for i in range(8,27,1)]
pop_address

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

In [87]:
pop_colname = []
for i in range(len(pop_address)):
    pop_colname.append(combined.columns[pop_address[i]]) 
pop_colname

['2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [88]:
combined.head(2)

,REGION,DIVISION,STATE,COUNTY,FIPS,State,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Est_Base_2000,Census_2010,Est_Base_2010,2001_per_change,2001_diff_change,2001cul_per_change,2001cul_diff_change,2002_per_change,2002_diff_change,2002cul_per_change,2002cul_diff_change,2003_per_change,2003_diff_change,2003cul_per_change,2003cul_diff_change,2004_per_change,2004_diff_change,2004cul_per_change,2004cul_diff_change,2005_per_change,2005_diff_change,2005cul_per_change,2005cul_diff_change,2006_per_change,2006_diff_change,2006cul_per_change,2006cul_diff_change,2007_per_change,2007_diff_change,2007cul_per_change,2007cul_diff_change,2008_per_change,2008_diff_change,2008cul_per_change,2008cul_diff_change,2009_per_change,2009_diff_change,2009cul_per_change,2009cul_diff_change,2010_per_change,2010_diff_change,2010cul_per_change,2010cul_diff_change,2011_per_change,2011_diff_change,2011cul_per_change,2011cul_diff_change,2012_per_change,2012_diff_change,2012cul_per_change,2012cul_diff_change,2013_per_change,2013_diff_change,2013cul_per_change,2013cul_diff_change,2014_per_change,2014_diff_change,2014cul_per_change,2014cul_diff_change,2015_per_change,2015_diff_change,2015cul_per_change,2015cul_diff_change,2016_per_change,2016_diff_change,2016cul_per_change,2016cul_diff_change,2017_per_change,2017_diff_change,2017cul_per_change,2017cul_diff_change,2018_per_change,2018_diff_change,2018cul_per_change,2018cul_diff_change,2019_per_change,2019_diff_change,2019cul_per_change,2019cul_diff_change
0,3,6,1,1,01001,Alabama,Autauga County,44021,44889,45909,46800,48366,49676,51328,52405,53277,54135,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0,43751,54571.0,54597.0,0.019718,868,0.019718,868,0.022723,1020,0.042889,1888,0.019408,891,0.063129,2779,0.033462,1566,0.098703,4345,0.027085,1310,0.128461,5655,0.033255,1652,0.165989,7307,0.020983,1077,0.190455,8384,0.016640,872,0.210263,9256,0.016105,858,0.229754,10114,0.011785,638.0,0.244247,10752.0,0.008289,454.0,0.254560,11206.0,-0.004943,-273.0,0.248359,10933.0,-0.004131,-227.0,0.243202,10706.0,0.003033,166.0,0.246973,10872.0,-0.000528,-29.0,0.246314,10843.0,0.006908,379.0,0.254924,11222.0,0.002661,147.0,0.258263,11369.0,0.002582,143.0,0.261512,11512.0,0.006050,336.0,0.269144,11848.0
1,3,6,1,3,01003,Alabama,Baldwin County,141342,144875,147957,151509,156266,162183,168121,172404,175827,179406,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0,140416,182265.0,182265.0,0.024996,3533,0.024996,3533,0.021274,3082,0.046801,6615,0.024007,3552,0.071932,10167,0.031397,4757,0.105588,14924,0.037865,5917,0.147451,20841,0.036613,5938,0.189462,26779,0.025476,4283,0.219765,31062,0.019855,3423,0.243983,34485,0.020355,3579,0.269304,38064,0.020657,3706.0,0.295524,41770.0,0.018819,3446.0,0.319905,45216.0,0.019227,3587.0,0.345283,48803.0,0.024928,4740.0,0.378819,53543.0,0.022054,4298.0,0.409227,57841.0,0.018857,3756.0,0.435801,61597.0,0.022972,4662.0,0.468785,66259.0,0.023699,4920.0,0.503594,71179.0,0.025099,5334.0,0.541332,76513.0,0.024691,5379.0,0.579389,81892.0


In [89]:
combined.columns.values

array(['REGION', 'DIVISION', 'STATE', 'COUNTY', 'FIPS', 'State', 'County',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 'Est_Base_2000', 'Census_2010', 'Est_Base_2010',
       '2001_per_change', '2001_diff_change', '2001cul_per_change',
       '2001cul_diff_change', '2002_per_change', '2002_diff_change',
       '2002cul_per_change', '2002cul_diff_change', '2003_per_change',
       '2003_diff_change', '2003cul_per_change', '2003cul_diff_change',
       '2004_per_change', '2004_diff_change', '2004cul_per_change',
       '2004cul_diff_change', '2005_per_change', '2005_diff_change',
       '2005cul_per_change', '2005cul_diff_change', '2006_per_change',
       '2006_diff_change', '2006cul_per_change', '2006cul_diff_change',
       '2007_per_change', '2007_diff_change', '2007cul_per_change',
       '2007cul_diff_change', '2008_per_change', '2008_diff_change',
       '2008cul_pe

In [90]:
keep = ['REGION', 'DIVISION', 'STATE', 'COUNTY', 'FIPS', 'State', 'County']
display(year)
display(diff_colname)
display(pop_colname)
display(change_colname)
display(cul_change_colname)
display(cul_diff_colname)

['2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

['2001_diff_change',
 '2002_diff_change',
 '2003_diff_change',
 '2004_diff_change',
 '2005_diff_change',
 '2006_diff_change',
 '2007_diff_change',
 '2008_diff_change',
 '2009_diff_change',
 '2010_diff_change',
 '2011_diff_change',
 '2012_diff_change',
 '2013_diff_change',
 '2014_diff_change',
 '2015_diff_change',
 '2016_diff_change',
 '2017_diff_change',
 '2018_diff_change',
 '2019_diff_change']

['2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

['2001_per_change',
 '2002_per_change',
 '2003_per_change',
 '2004_per_change',
 '2005_per_change',
 '2006_per_change',
 '2007_per_change',
 '2008_per_change',
 '2009_per_change',
 '2010_per_change',
 '2011_per_change',
 '2012_per_change',
 '2013_per_change',
 '2014_per_change',
 '2015_per_change',
 '2016_per_change',
 '2017_per_change',
 '2018_per_change',
 '2019_per_change']

['2001cul_per_change',
 '2002cul_per_change',
 '2003cul_per_change',
 '2004cul_per_change',
 '2005cul_per_change',
 '2006cul_per_change',
 '2007cul_per_change',
 '2008cul_per_change',
 '2009cul_per_change',
 '2010cul_per_change',
 '2011cul_per_change',
 '2012cul_per_change',
 '2013cul_per_change',
 '2014cul_per_change',
 '2015cul_per_change',
 '2016cul_per_change',
 '2017cul_per_change',
 '2018cul_per_change',
 '2019cul_per_change']

['2001cul_diff_change',
 '2002cul_diff_change',
 '2003cul_diff_change',
 '2004cul_diff_change',
 '2005cul_diff_change',
 '2006cul_diff_change',
 '2007cul_diff_change',
 '2008cul_diff_change',
 '2009cul_diff_change',
 '2010cul_diff_change',
 '2011cul_diff_change',
 '2012cul_diff_change',
 '2013cul_diff_change',
 '2014cul_diff_change',
 '2015cul_diff_change',
 '2016cul_diff_change',
 '2017cul_diff_change',
 '2018cul_diff_change',
 '2019cul_diff_change']

In [91]:
# melt the data for Tableau use
molten_parts=[]
for i in range(0,19,1):
    #print(i)
    empty=pd.DataFrame()
    empty[keep]=combined[keep]
    empty['year']=year[i]
    empty['pop']=combined[pop_colname[i]]
    empty['change']=combined[change_colname[i]]
    empty['diff']=combined[diff_colname[i]]
    empty['cul_change']=combined[cul_change_colname[i]]
    empty['cul_diff']=combined[cul_diff_colname[i]]
    molten_parts.append(empty)
    #hold=pd.concat[hold,empty]
melt=pd.concat(molten_parts)
display(melt)
    

,REGION,DIVISION,STATE,COUNTY,FIPS,State,County,year,pop,change,diff,cul_change,cul_diff
0,3,6,1,1,01001,Alabama,Autauga County,2001,44889.0,0.019718,868.0,0.019718,868.0
1,3,6,1,3,01003,Alabama,Baldwin County,2001,144875.0,0.024996,3533.0,0.024996,3533.0
2,3,6,1,5,01005,Alabama,Barbour County,2001,28863.0,-0.005239,-152.0,-0.005239,-152.0
3,3,6,1,7,01007,Alabama,Bibb County,2001,21028.0,0.055994,1115.0,0.055994,1115.0
4,3,6,1,9,01009,Alabama,Blount County,2001,51845.0,0.014440,738.0,0.014440,738.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,4,8,56,37,56037,Wyoming,Sweetwater County,2019,42343.0,-0.012016,-515.0,0.127583,4791.0
3137,4,8,56,39,56039,Wyoming,Teton County,2019,23464.0,0.008380,195.0,0.276536,5083.0
3138,4,8,56,41,56041,Wyoming,Uinta County,2019,20226.0,-0.003253,-66.0,0.028476,560.0
3139,4,8,56,43,56043,Wyoming,Washakie County,2019,7805.0,-0.009141,-72.0,-0.054169,-447.0


In [92]:
state_year_tot = melt[['State','year','pop']].groupby(['State','year']).sum().reset_index()
state_year_tot

,State,year,pop
0,Alabama,2001,4467634.0
1,Alabama,2002,4480089.0
2,Alabama,2003,4503491.0
3,Alabama,2004,4530729.0
4,Alabama,2005,4569805.0
...,...,...,...
964,Wyoming,2015,585613.0
965,Wyoming,2016,584215.0
966,Wyoming,2017,578931.0
967,Wyoming,2018,577601.0


In [93]:
state_year_tot['year']=pd.to_datetime(state_year_tot['year'])

In [94]:
state_year_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   State   969 non-null    object        
 1   year    969 non-null    datetime64[ns]
 2   pop     969 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 22.8+ KB


#### 6.3 export df

In [95]:
state_year_tot.to_csv('state_year_tot.csv', index=False)

In [96]:
melt.to_csv('year_to_year.csv', index=False)

<a href='#TOB'>Back to table of content</a>  